通过历史数据评估阳床树脂再生能力。

因所填表格不满足数据库第一范式，所以需要处理数据的大量工作。

从更换树脂当天至未来的某一天，当原树脂洗床所多消耗酸的成本可以覆盖更换树脂的成本时，就值得更换树脂。

清洗单个单元格

In [6]:
import pandas as pd
import re
import calendar
import os
from time import sleep
from tqdm.auto import tqdm

# 初始化文件
if os.path.exists('RegenerationInfo.csv'):
    os.remove('RegenerationInfo.csv')

valid_columns = [2,5,8]
years = [2019,2020,2021,2022]

for year in tqdm(years,desc="year"):
    if (year < 2021):
        valid_index1 = 25
        valid_index2 = 26
    else:
        valid_index1 = 27
        valid_index2 = 28

    # 遍历路径文件
    for month in tqdm(range(1,13,1),desc="month"):
        file_name = '干熄焦生产记录台账/{0}/干熄焦生产记录台账（{1}月）.xlsx'.format(year,month)
        #file_date_info = re.findall(r'\d{1,}', file_name) # 提取路径文件信息(年、月）
        #month = file_date_info[1]
        # year = file_date_info[0]

        # 返回（当月总周数，当月总天数）
        month_range = calendar.monthrange(int(year),int(month))
        
        # 遍历表单
        for day in range(1,month_range[1],1):  # 从第1天开始至当月最后一天
            try:
                ws_data = pd.read_excel(file_name, sheet_name = "{}日".format(day), header=None) # 读取当日表单
            except FileNotFoundError as file_error:
                print(file_error)
            ws_data_regeneration = ws_data.iloc[valid_index1:valid_index2,[2,5,8]].astype(str) # 只保留包含洗床单元格的数据
            
            # 构建日期DataFrame
            regenerate_date = {'date':["{0}-{1}-{2}".format(year, month, day)]} # 再生日期信息
            regeneration_date = pd.DataFrame(data=regenerate_date)
        
            # 对单元格遍历，拆分、分割
            for valid_column in valid_columns:
                ws_regeneration_split_cell = ws_data_regeneration[valid_column].T.str.split(r'\s+', expand=True) # 转置,（以至少1个空格)分割
                # print(ws_regeneration_split_cell)
            
                # 根据拆分的单元格长度，遍历追加入文件
                for l in range(int(ws_regeneration_split_cell.size)):
                    ws_regeneration_split_cell[l] =  ws_regeneration_split_cell[l].astype(str)
                        
                    # 重建洗床信息DataFrame
                    ws_regeneration_split_cell_value = ws_regeneration_split_cell[l].values
                    ws_regeneration_split_cell_value_df = pd.DataFrame(ws_regeneration_split_cell_value)
                        
                    frames = [regeneration_date, ws_regeneration_split_cell_value_df]
                    result = pd.concat(frames,join="outer",axis=1, ignore_index=True)
                    
                    result.to_csv("RegenerationInfo.csv",mode='a',header=False, index=False)
                

rename = pd.read_csv("RegenerationInfo.csv")
rename.rename({0:'date',1:'regeneration'},axis=0, inplace=True)

year:   0%|          | 0/4 [00:00<?, ?it/s]

month:   0%|          | 0/12 [00:00<?, ?it/s]

month:   0%|          | 0/12 [00:00<?, ?it/s]

month:   0%|          | 0/12 [00:00<?, ?it/s]

month:   0%|          | 0/12 [00:00<?, ?it/s]

[Errno 2] No such file or directory: '干熄焦生产记录台账/2022/干熄焦生产记录台账（1月）.xlsx'
[Errno 2] No such file or directory: '干熄焦生产记录台账/2022/干熄焦生产记录台账（1月）.xlsx'
[Errno 2] No such file or directory: '干熄焦生产记录台账/2022/干熄焦生产记录台账（1月）.xlsx'
[Errno 2] No such file or directory: '干熄焦生产记录台账/2022/干熄焦生产记录台账（1月）.xlsx'
[Errno 2] No such file or directory: '干熄焦生产记录台账/2022/干熄焦生产记录台账（1月）.xlsx'
[Errno 2] No such file or directory: '干熄焦生产记录台账/2022/干熄焦生产记录台账（1月）.xlsx'
[Errno 2] No such file or directory: '干熄焦生产记录台账/2022/干熄焦生产记录台账（1月）.xlsx'
[Errno 2] No such file or directory: '干熄焦生产记录台账/2022/干熄焦生产记录台账（1月）.xlsx'
[Errno 2] No such file or directory: '干熄焦生产记录台账/2022/干熄焦生产记录台账（1月）.xlsx'
[Errno 2] No such file or directory: '干熄焦生产记录台账/2022/干熄焦生产记录台账（1月）.xlsx'
[Errno 2] No such file or directory: '干熄焦生产记录台账/2022/干熄焦生产记录台账（1月）.xlsx'
[Errno 2] No such file or directory: '干熄焦生产记录台账/2022/干熄焦生产记录台账（1月）.xlsx'
[Errno 2] No such file or directory: '干熄焦生产记录台账/2022/干熄焦生产记录台账（1月）.xlsx'
[Errno 2] No such file or directory: '干熄焦生产记录台账/202

正则匹配2#阳床洗床时间

In [ ]:
rg_info = pd.read_csv("RegenerationInfo.csv")

fcation_resin_2 = re.compile(r'\d{1,}:\d{1,}分再生2#阳床')

rg_info.Series.str.match(r'\d{1,}:\d{1,}分再生2#阳床')

# for m in rg_info.index:
#     cation_resin_2 = re.search(fcation_resin_2, rg_info[0])
#     print(yang2)